In [ ]:
!pip install tensorflow==2.6.0 keras==2.6.0

ERROR: Could not find a version that satisfies the requirement tensorflow==2.6.0 (from versions: 2.8.0rc0, 2.8.0rc1, 2.8.0, 2.8.1, 2.8.2, 2.8.3, 2.8.4, 2.9.0rc0, 2.9.0rc1, 2.9.0rc2, 2.9.0, 2.9.1, 2.9.2, 2.9.3, 2.10.0rc0, 2.10.0rc1, 2.10.0rc2, 2.10.0rc3, 2.10.0, 2.10.1, 2.11.0rc0, 2.11.0rc1, 2.11.0rc2, 2.11.0, 2.11.1, 2.12.0rc0, 2.12.0rc1, 2.12.0, 2.12.1, 2.13.0rc0, 2.13.0rc1, 2.13.0rc2, 2.13.0, 2.13.1, 2.14.0rc0, 2.14.0rc1, 2.14.0)
ERROR: No matching distribution found for tensorflow==2.6.0


In [ ]:
!pip install --upgrade onnx

In [ ]:
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
print(tf.__version__)
import torch
import keras
import gc

2.14.0


In [ ]:
!pip install keras_cv_attention_models


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 773.2/773.2 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.4/53.4 kB 7.7 MB/s eta 0:00:00


In [ ]:
!pip install tflite_runtime
!pip install tflite

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 11.5 MB/s eta 0:00:00


In [ ]:
from keras_cv_attention_models import efficientvit

# Will download and load pretrained imagenet weights.
model = efficientvit.EfficientViT_B1(pretrained="imagenet")

ModuleNotFoundError: ignored

In [ ]:
import h5py
import re

def get_tensorflow_version(h5):
    with h5py.File(h5, 'r') as f:
        metadata = f.attrs['model_config']
        return metadata

print(get_tensorflow_version('efficientvit_b0_224_imagenet.h5'))


In [ ]:
! edgetpu_compiler  quantized_spatial_extractor.tflite -sa

Edge TPU Compiler version 16.0.384591198
Started a compilation timeout timer of 180 seconds.

Model compiled successfully in 32789 ms.

Input model: quantized_spatial_extractor.tflite
Input size: 5.82MiB
Output model: quantized_spatial_extractor_edgetpu.tflite
Output size: 11.13MiB
On-chip memory used for caching model parameters: 5.97MiB
On-chip memory remaining for caching model parameters: 0.00B
Off-chip memory used for streaming uncached model parameters: 2.05MiB
Number of Edge TPU subgraphs: 3
Total number of operations: 1055
Operation log: quantized_spatial_extractor_edgetpu.log

Model successfully compiled but not all operations are supported by the Edge TPU. A percentage of the model will instead run on the CPU, which is slower. If possible, consider updating your model to use only operations supported by the Edge TPU. For details, visit g.co/coral/model-reqs.
Number of operations that will run on Edge TPU: 1051
Number of operations that will run on CPU: 4

Operator            

In [ ]:
! edgetpu_compiler  quantized_temporal_extractor.tflite  quantized_spatial_extractor.tflite -sa

Edge TPU Compiler version 16.0.384591198
Started a compilation timeout timer of 180 seconds.

Models compiled successfully in 32686 ms.

Input model: quantized_temporal_extractor.tflite
Input size: 2.46MiB
Output model: quantized_temporal_extractor_edgetpu.tflite
Output size: 3.26MiB
On-chip memory used for caching model parameters: 2.34MiB
On-chip memory remaining for caching model parameters: 0.00B
Off-chip memory used for streaming uncached model parameters: 182.06KiB
Number of Edge TPU subgraphs: 4
Total number of operations: 221
Operation log: quantized_temporal_extractor_edgetpu.log

Model successfully compiled but not all operations are supported by the Edge TPU. A percentage of the model will instead run on the CPU, which is slower. If possible, consider updating your model to use only operations supported by the Edge TPU. For details, visit g.co/coral/model-reqs.
Number of operations that will run on Edge TPU: 208
Number of operations that will run on CPU: 13

Operator        

In [ ]:
layers = ""
for i in range(12):
    layers += f"transformer_block_{i}/layer_normalization_1/batchnorm/sub,transformer_block_{i}/layer_normalization_1/batchnorm/add_1,Tensordot;transformer_block_{i}/dense/StatefulPartitionedCall/StatefulPartitionedCall/Tensordot;BiasAdd;transformer_block_{i}/dense/StatefulPartitionedCall/StatefulPartitionedCall/BiasAdd,tfl.dequantize,Gelu/Pow;transformer_block_{i}/dense/StatefulPartitionedCall/StatefulPartitionedCall/Gelu/Pow,tfl.quantize,Gelu/mul_1;transformer_block_{i}/dense/StatefulPartitionedCall/StatefulPartitionedCall/Gelu/mul_1,Gelu/add;transformer_block_{i}/dense/StatefulPartitionedCall/StatefulPartitionedCall/Gelu/add,Gelu/mul_2;transformer_block_{i}/dense/StatefulPartitionedCall/StatefulPartitionedCall/Gelu/mul_2,Gelu/Tanh;transformer_block_{i}/dense/StatefulPartitionedCall/StatefulPartitionedCall/Gelu/Tanh,Gelu/add_1;transformer_block_{i}/dense/StatefulPartitionedCall/StatefulPartitionedCall/Gelu/add_1"

    # Add a comma if it's not the last iteration
    if i < 11:
        layers += ","

command = f"edgetpu_compiler quantized_spatial_extractor.tflite -as -i \"{layers}\""
print(command)

edgetpu_compiler quantized_spatial_extractor.tflite -as -i "transformer_block_0/layer_normalization_1/batchnorm/sub,transformer_block_0/layer_normalization_1/batchnorm/add_1,Tensordot;transformer_block_0/dense/StatefulPartitionedCall/StatefulPartitionedCall/Tensordot;BiasAdd;transformer_block_0/dense/StatefulPartitionedCall/StatefulPartitionedCall/BiasAdd,tfl.dequantize,Gelu/Pow;transformer_block_0/dense/StatefulPartitionedCall/StatefulPartitionedCall/Gelu/Pow,tfl.quantize,Gelu/mul_1;transformer_block_0/dense/StatefulPartitionedCall/StatefulPartitionedCall/Gelu/mul_1,Gelu/add;transformer_block_0/dense/StatefulPartitionedCall/StatefulPartitionedCall/Gelu/add,Gelu/mul_2;transformer_block_0/dense/StatefulPartitionedCall/StatefulPartitionedCall/Gelu/mul_2,Gelu/Tanh;transformer_block_0/dense/StatefulPartitionedCall/StatefulPartitionedCall/Gelu/Tanh,Gelu/add_1;transformer_block_0/dense/StatefulPartitionedCall/StatefulPartitionedCall/Gelu/add_1,transformer_block_1/layer_normalization_1/batchn

In [ ]:

import tflite_runtime.interpreter as tflite

interpreter = tflite.Interpreter(model_path='quantized_spatial_extractor.tflite')
all_layers_details = interpreter.get_tensor_details()
#print(len(all_layers_details))
# Calculate the range for the layers
offset = 7
size_of_transformer_block = 81
start_index = -offset - 35
end_index = -offset - 22
res = ""
for i in range(0, 12):


  for layer_index in range(start_index - size_of_transformer_block*i, end_index -  size_of_transformer_block*i + 1):
      layer = all_layers_details[layer_index]
      res += (layer['name'] +',')
      #print(layer['name'] +',', end='')
print(res)

model_242/tf.linalg.matmul_137/MatMul19,model_242/tf.linalg.matmul_137/MatMul20,model_242/tf.linalg.matmul_137/MatMul21,model_242/tf.linalg.matmul_137/MatMul22,model_242/tf.linalg.matmul_137/MatMul23,model_242/tf.compat.v1.transpose_133/transpose,model_242/tf.linalg.matmul_139/MatMul;model_242/tf.reshape_74/Reshape,model_242/tf.linalg.matmul_139/MatMul;model_242/tf.math.add_115/Add;model_242/tf.linalg.matmul_143/MatMul;model_242/tf.math.add_115/Add/y,model_242/tf.math.add_115/Add;model_242/tf.linalg.matmul_143/MatMul;model_242/tf.linalg.matmul_139/MatMul;model_242/tf.math.add_115/Add/y,model_242/tf.math.add_116/Add,model_242/tf.math.reduce_mean_46/Mean,model_242/tf.math.subtract_25/Sub,model_242/tf.math.multiply_400/Mul,model_242/tf.math.reduce_mean_47/Mean,model_242/tf.linalg.matmul_125/MatMul14,model_242/tf.linalg.matmul_125/MatMul15,model_242/tf.linalg.matmul_125/MatMul16,model_242/tf.linalg.matmul_125/MatMul17,model_242/tf.linalg.matmul_125/MatMul18,model_242/tf.compat.v1.transpose

In [ ]:
!pip install tflite-runtime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 10.2 MB/s eta 0:00:00


In [ ]:
import tflite_runtime.interpreter as tflite

interpreter = tflite.Interpreter(model_path='quantized_spatial_extractor.tflite')
all_layers_details = interpreter.get_tensor_details()

for layer in all_layers_details:
  print(layer['name'])

In [ ]:
! apt-get remove edgetpu-compiler

! cp -r /content/drive/MyDrive/compiler   /content
! chmod +x compiler/edgetpu_compiler
! chmod -R +x compiler/edgetpu_compiler_bin

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
E: Unable to locate package edgetpu-compiler
^C
chmod: cannot access 'compiler/edgetpu_compiler_bin': No such file or directory


In [ ]:
! curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -

! echo "deb https://packages.cloud.google.com/apt coral-edgetpu-stable main" | sudo tee /etc/apt/sources.list.d/coral-edgetpu.list

! sudo apt-get update

! sudo apt-get install edgetpu-compiler

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2659  100  2659    0     0  20760      0 --:--:-- --:--:-- --:--:-- 20937
OK
deb https://packages.cloud.google.com/apt coral-edgetpu-stable main
Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:2 https://packages.cloud.google.com/apt coral-edgetpu-stable InRelease [6,332 B]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:7 https://packages.cloud.google.com/apt coral-edgetpu-stable/main amd64 Packages [2,317 B]
Hit:8 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:9 https://developer.download.nvidia.c

In [ ]:
# Load the TFLite model and allocate tensors
interpreter = tf.lite.Interpreter(model_path="model_tflite.tflite")
interpreter.allocate_tensors()

# Get input and output tensors
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print(input_details)
print(output_details)

interpreter.invoke()
# Load the TFLite model and allocate tensors
interpreter2 = tf.lite.Interpreter(model_path="PROC_CLS.tflite")
interpreter2.allocate_tensors()

# Get input and output tensors
input_details = interpreter2.get_input_details()
output_details = interpreter2.get_output_details()

print(input_details)
print(output_details)

interpreter2.invoke()


In [ ]:
import h5py
import re

def get_tensorflow_version(h5):
    with h5py.File(h5, 'r') as f:
        model_config = f.attrs.get('model_config')
        if model_config is None:
            return "No Version"
        else:
            model_config_str = model_config
            version = re.search(r'"keras_version":\s*"([^"]*)', model_config_str)
            if version:
                return version.group(1)
            else:
                return "No Version"

print(get_tensorflow_version('efficientvit_b0_224_imagenet.h5'))

No Version


In [ ]:
%env TFLITE_FILE=model_tflite.tflite

env: TFLITE_FILE=model_tflite.tflite


In [ ]:

import keras_cv_attention_models
import numpy as np

import tensorflow as tf
from tensorflow import keras
from keras.models import load_model
from keras_cv_attention_models import efficientvit_b, mobilevit, common_layers

def modify_layers(layer):
    layer_type = type(layer).__name__
    print(layer_type)
    if hasattr(layer, 'activation'):
        print(layer_type, layer.activation.__name__)
        if layer.activation.__name__ == 'gelu':

            target_activation = "gelu/quick"
            print("replaced gelu")
            layer._name = "REPL_GELU"
            if target_activation == "gelu/quick":

                layer.activation = tf.keras.activations.relu
                layer.activation.__name__ = "gelu/quick"
            else:
                layer.activation = tf.keras.activations.relu
                layer.activation.__name__ = "gelu/quick"
                # layer.activation = lambda xx: functional.gelu(xx, approximate=True)

    if hasattr(layer, 'layers'):
        for sub_layer in layer.layers:
            modify_layers(sub_layer)


def SpatialExtractorDeiT() -> keras.Model:
    # Create a new model with the desired layers
    model_gcs_path = "https://tfhub.dev/sayakpaul/deit_tiny_distilled_patch16_224_fe/1/uncompressed"
    # vit = hub.KerasLayer(model_gcs_path, trainable=False)
    vit = keras.models.load_model("deit_tiny_distilled_patch16_224_fe")

    vit.trainable = False

    for layer in vit.layers:
        modify_layers(layer)
    # vit.build(input_shape=(1, 224, 224, 3))

    vit.save("test")
    vit = load_model("test")

    vit.summary()
    # vit = keras.models.load_model("deit_tiny_distilled_patch16_224")
    # vit = model_surgery.convert_gelu_to_approximate(vit)
    inp = keras.layers.Input(shape=(224, 224, 3))
    x = keras.layers.experimental.preprocessing.Rescaling(1. / 255, input_shape=input_shape)(inp)
    x = PreprocessTFLayer()(x)
    vec = vit(x)
    model = keras.models.Model(inputs=inp, outputs=vec[0])
    model.summary()
    return model


SpatialExtractorDeiT()

Sequential
Conv2D
Conv2D linear
Reshape
Functional
InputLayer
LayerNormalization
TFViTAttention
Add
LayerNormalization
Dense
Dense gelu
replaced gelu
Dropout
Dense
Dense linear
Dropout
Add
Functional
InputLayer
LayerNormalization
TFViTAttention
Add
LayerNormalization
Dense
Dense gelu
replaced gelu
Dropout
Dense
Dense linear
Dropout
Add
Functional
InputLayer
LayerNormalization
TFViTAttention
Add
LayerNormalization
Dense
Dense gelu
replaced gelu
Dropout
Dense
Dense linear
Dropout
Add
Functional
InputLayer
LayerNormalization
TFViTAttention
Add
LayerNormalization
Dense
Dense gelu
replaced gelu
Dropout
Dense
Dense linear
Dropout
Add
Functional
InputLayer
LayerNormalization
TFViTAttention
Add
LayerNormalization
Dense
Dense gelu
replaced gelu
Dropout
Dense
Dense linear
Dropout
Add
Functional
InputLayer
LayerNormalization
TFViTAttention
Add
LayerNormalization
Dense
Dense gelu
replaced gelu
Dropout
Dense
Dense linear
Dropout
Add
Functional
InputLayer
LayerNormalization
TFViTAttention
Add
LayerN